<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/tokenizer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装依赖

In [ ]:
!pip install datasets
!pip install transformers

# 使用原始 roberta 进行分词

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

robert_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

# prepare input
texts = [
    "Replace me by any text you'd like.",
    "他用拂尘轻轻抚过窗棂，擦去了昨夜雨水留下的痕迹。",
    "落日余晖透过稠密的树林，洒在湖面上，形成美丽的倒影。",
    "前瞻：迈阿密三巨头无担心必要,雄鹿指望出奇制胜",
    "富力桃园8月28日举办社区商业价值探寻投资论坛"
]
tokens_list = [robert_tokenizer.tokenize(text) for text in texts]
[print(token) for token in tokens_list]


['▁Re', 'place', '▁me', '▁by', '▁any', '▁text', '▁you', "'", 'd', '▁like', '.']
['▁他', '用', '拂', '尘', '轻轻', '抚', '过', '窗', '棂', ',', '擦', '去了', '昨', '夜', '雨', '水', '留下', '的', '痕迹', '。']
['▁', '落', '日', '余', '晖', '透过', '稠', '密', '的', '树', '林', ',', '洒', '在', '湖', '面', '上', ',', '形成', '美丽的', '倒', '影', '。']
['▁前', '瞻', ':', '迈', '阿', '密', '三', '巨头', '无', '担心', '必要', ',', '雄', '鹿', '指', '望', '出', '奇', '制', '胜']
['▁', '富', '力', '桃', '园', '8', '月', '28', '日', '举办', '社区', '商业', '价值', '探', '寻', '投资', '论坛']


[None, None, None, None, None]

从分词结果来看，一些词被正确地分在了一起，如：“轻轻”、“美丽的”、“巨头”、“举办” 等等。

有些词，如“窗棂”、“落日”、“迈阿密”、“富力桃园” 等则没有被正确拆分。

下面我们使用新的数据集对 robert 重新训练，然后看看训练后的分词效果。

# 使用新的数据集重新训练

## 训练第一步：加载数据集

In [ ]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = dataset = load_dataset("seamew/THUCNews")
print(raw_datasets)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})


seamew/THUCNews 数据集可以在 hugging face 上找到。训练数据5w条，每条句子也比较短。选择这样一个比较小的数据集是因为省机器资源，并且训练很快。

## 训练第二步：加载原模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

old_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

## 训练第三步：在原模型上跑数据集进行训练，结束后将新模型保存到本地

In [ ]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["text"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 50000)
tokenizer.save_pretrained("ch-roberta-thucnews")

('ch-roberta-thucnews/tokenizer_config.json',
 'ch-roberta-thucnews/special_tokens_map.json',
 'ch-roberta-thucnews/tokenizer.json')

training_corpus 是一个生成器表达式，将原始数据集中的训练数据按照每1000个为一组进行了分割。


**使用生成器可以避免将整个数据集一次性加载到内存中**，因为生成器会逐个地生成数据项并返回，而不是像列表一样一次性将所有数据都放入内存中。在对数据进行处理时，生成器使得我们能够一次性只处理一部分数据，从而减轻了 RAM 内存的压力。

## 训练第四步：验证

In [ ]:
texts = [
    "Replace me by any text you'd like.",
    "他用拂尘轻轻抚过窗棂，擦去了昨夜雨水留下的痕迹。",
    "落日余晖透过稠密的树林，洒在湖面上，形成美丽的倒影。",
    "前瞻：迈阿密三巨头无担心必要,雄鹿指望出奇制胜",
    "富力桃园8月28日举办社区商业价值探寻投资论坛"
]
tokens_list = [tokenizer.tokenize(text) for text in texts]
[print(token) for token in tokens_list]

['▁R', 'e', 'p', 'la', 'ce', '▁', 'me', '▁', 'by', '▁', 'any', '▁', 't', 'e', 'x', 't', '▁', 'y', 'o', 'u', "'", 'd', '▁', 'li', 'ke', '.']
['▁', '他用', '拂', '尘', '轻', '轻', '抚', '过', '窗', '棂', ',', '擦', '去', '了', '昨', '夜', '雨', '水', '留下', '的', '痕迹', '。']
['▁', '落', '日', '余', '晖', '透过', '稠', '密', '的', '树林', ',', '洒', '在', '湖', '面', '上', ',', '形成美丽', '的', '倒影', '。']
['▁前瞻:', '迈阿密三巨头', '无', '担心', '必要', ',雄鹿', '指望', '出奇制胜']
['▁富力桃园8月28日', '举办', '社区商业', '价值', '探寻', '投资', '论坛']


[None, None, None, None, None]

可以看到，重新训练后，分词有所不同。

例如：“前瞻”、“迈阿密三巨头”、“社区商业”、“富力桃园” 等可以正确拆分。

当然，这里只是举例说明。如果想要更好的效果，还需要使用更大的数据集进行训练，“大力出奇迹”。

In [ ]:
ls -ahl ./ch-roberta-thucnews/

total 3.5M
drwxr-xr-x 2 root root 4.0K Jun  1 06:06 ./
drwxr-xr-x 1 root root 4.0K Jun  1 06:06 ../
-rw-r--r-- 1 root root  280 Jun  1 06:31 special_tokens_map.json
-rw-r--r-- 1 root root  419 Jun  1 06:31 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Jun  1 06:31 tokenizer.json


## 训练第五步（可选）：授权 hugging face，并将模型保存到自己的空间

1. token可以从 hugging face 个人信息 setting 中拿到。需要新增一个有写入权限的 token

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

2. 将模型推送到 hugging face

In [ ]:
tokenizer.push_to_hub("ch-roberta-thucnews")

3. 验证保存到 hugging face 的新模型，注意要在模型前面加上自己的空间名称

In [ ]:
from transformers import AutoTokenizer

saved_tokenizer = AutoTokenizer.from_pretrained('erberry/ch-roberta-thucnews')
# prepare input
texts = [
    "Replace me by any text you'd like.",
    "他用拂尘轻轻抚过窗棂，擦去了昨夜雨水留下的痕迹。",
    "落日余晖透过稠密的树林，洒在湖面上，形成美丽的倒影。",
    "前瞻：迈阿密三巨头无担心必要,雄鹿指望出奇制胜",
    "富力桃园8月28日举办社区商业价值探寻投资论坛"
]
tokens_list = [saved_tokenizer.tokenize(text) for text in texts]
[print(token) for token in tokens_list]

['▁R', 'e', 'p', 'la', 'ce', '▁', 'me', '▁', 'by', '▁', 'any', '▁', 't', 'e', 'x', 't', '▁', 'y', 'o', 'u', "'", 'd', '▁', 'li', 'ke', '.']
['▁', '他用', '拂', '尘', '轻', '轻', '抚', '过', '窗', '棂', ',', '擦', '去', '了', '昨', '夜', '雨', '水', '留下', '的', '痕迹', '。']
['▁', '落', '日', '余', '晖', '透过', '稠', '密', '的', '树林', ',', '洒', '在', '湖', '面', '上', ',', '形成美丽', '的', '倒影', '。']
['▁前瞻:', '迈阿密三巨头', '无', '担心', '必要', ',雄鹿', '指望', '出奇制胜']
['▁富力桃园8月28日', '举办', '社区商业', '价值', '探寻', '投资', '论坛']


[None, None, None, None, None]